In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import plotly.express as px
import plotly
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

plotly.offline.init_notebook_mode()
%matplotlib inline
%config InlineBackend.figure_format='retina'

: 

## Load data

[Fashion MNIST](https://keras.io/api/datasets/fashion_mnist/) data is loaded using `keras.datasets`.  
The dataset consists of both training and testing sets. There are 60,000 images for training and 10,000 for testing.  

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

: 

In [ ]:
print(f"Shape of training data: {X_train.shape}")
print(f"Shape of testing data: {X_test.shape}")

: 

As the label in the train and test sets are 0-9 integers, we need to create a `category_list` for lookup between the integer labels and the true category in text.  
<br>
Based on the dataset description, the labels of the categories are as follows:

| Category | Label |
|:-|:-:|
|T-shirt/Top|0|
|Trouser|1|
|Pullover|2|
|Dress|3|
|Coat|4|
|Sandal|5|
|Shirt|6|
|Sneaker|7|
|Bag|8|
|Ankle Boot|9|

Therefore, we create the following list contains the *categories* as the list items and the *labels* as the item indices.


In [ ]:
category_list = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

: 

## Visualize sample images

Randomly pick 10 images from the training set and show them in two rows of 5

In [ ]:
# Pick 10 random indices
idx = np.random.randint(0, len(X_train), 10)
facet_labels = y_train[idx]

# Work-around to get titles in correct order for plotting in plotly
facet_col_wrap = len(idx) // 2
facet_labels = np.hstack((facet_labels[facet_col_wrap:], facet_labels[:facet_col_wrap]))

# Plot images
fig = px.imshow(X_train[idx], binary_string=True, facet_col=0,
                facet_col_wrap=facet_col_wrap,
                labels={'facet_col':'category'})

# Show category titles
for i, label in enumerate(facet_labels):
    fig.layout.annotations[i]['text'] = f"category = {category_list[label]}"

fig.show()

: 

## Data Preprocessing

Add the *channel* dimension to the images

In [ ]:
X_train = X_train[:, :, :, np.newaxis]
X_test = X_test[:, :, :, np.newaxis]

print(f"Shape of training data: {X_train.shape}")
print(f"Shape of testing data: {X_test.shape}")

: 

## Build and train convolutional network for image classification

### Model architecture

The model comprises of <font color='blue'>3 convolutional layers</font>, <font color='purple'>2 max pooling layers</font>, followed by <font color='green'>a flatten layer</font> and <font color='orange'>a dense layer</font>.  

In [ ]:
img = Image.open("../image/model_architecture.png")
plt.figure(figsize=(10, 10))
fig = plt.imshow(img)
plt.axis('off')
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)

: 

### Build model

Build a CNN with the above architecture but has `Rescaling` layers to normalize the pixel values.

In [ ]:
tf.random.set_seed(42)

model = keras.Sequential([
    keras.layers.Rescaling(scale=1./255, input_shape=(28, 28, 1)),
    keras.layers.Conv2D(8, 3, activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(16, 3, activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, 3, activation="relu"),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation="softmax")
])

model.summary()

: 

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

: 

### Train model

In [ ]:
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    shuffle=True,
                    epochs=3,
                    batch_size=16)

: 

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accuracy)+1)

# Plot Accuracy
plt.figure(figsize=(5, 3))
plt.plot(epochs, accuracy)
plt.plot(epochs, val_accuracy)
plt.xlabel('Epochs')
plt.xticks(epochs) 
plt.title('Accuracy')
plt.legend(['train', 'validation'])
plt.show()

# Plot Loss
plt.figure(figsize=(5, 3))
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.xlabel('Epochs')
plt.xticks(epochs) 
plt.title('Loss')
plt.legend(['train', 'validation'])
plt.show()

: 

## Evaluate model on test set

### Accuracy

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", round(accuracy, 3))

: 

### Precision, Recall and F1 Score

We use `classification_report` to evaluate the `precision`, `recall` and `f1_score`.  
Under the hood, the formulas of these metrics are as follows:  
<br>
$$
 precision = \frac{TP}{TP + FP} \\
 recall = \frac{TP}{TP + FN}\\
 F_1 = \frac{2}{\frac{1}{precision} + \frac{1}{recall}} = 2\times \frac{precision\times recall}{precision + recall}
$$
<br>
where TP = true positives, FP = false positives, and FN = false negatives

In [ ]:
y_pred = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred, axis=-1)

print(classification_report(y_test, y_pred, target_names=category_list))

: 

## Save & load model

In [ ]:
model.save("../model/mnist_fashion_saved_model")

: 

In [ ]:
loaded_model = tf.keras.models.load_model("../model/mnist_fashion_saved_model")
loaded_model.summary()

: 

: 